In [22]:
import random
import plotly
import warnings
import keras as k
import numpy as np
import pandas as pd
import seaborn as sns
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import roc_curve, roc_auc_score
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from models.linear_regression.linear_regression import LinearRegression as LR

import config as c
from sklearn.model_selection import train_test_split
from pipeline.Layer import Layer
from utils.cuda import turn_off_gpu
from models.svm.svm import SVM
from utils.metrics import roc_auc_score_at_K
from utils.preprocess import preprocess, add_columns, reset_averages
from models.keras_dense_classifier.keras_dense_classifier import KerasDenseClassifier as KDC
from visualization.utils import plot_correlation_matrix, plot_scatterplot_matrix

#turn_off_gpu()
init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')
plt.rcParams ['figure.figsize'] = (13,8)
sns.set()
%config InlineBackend.figure_format = 'retina'

ImportError: cannot import name 'reset_averages' from 'utils.preprocess' (C:\Users\Kiril\Documents\GitHub\IDAO-2020-FinalRound\utils\preprocess.py)

In [35]:
df = pd.read_csv('train.csv', sep=',')
pd.set_option('display.max_columns', 500)
df = add_columns(df)
df = preprocess(df)
df

card_id  target  addr_region_reg  addr_region_fact  \
0       cid_10620       1              107               107   
1      cid_105724       0                9                 9   
2      cid_101410       1              109               109   
3       cid_38961       0               66                66   
4       cid_57462       0               16                16   
...           ...     ...              ...               ...   
89995   cid_34679       0                5                 5   
89996   cid_83577       1               56                56   
89997  cid_127181       0              109               109   
89998  cid_113463       0               60                60   
89999  cid_102137       0               56                56   

       sas_limit_after_003_amt  sas_limit_last_amt  \
0                            1                   1   
1                            3                   3   
2                            1                   1   
3                            3                   3   
4                            0                   0   
...                        ...                 ...   
89995                        4                   4   
89996                        2                   2   
89997                        2                   2   
89998                        0                   0   
89999                        6                   6   

       clnt_income_month_avg_net_amt  clnt_expense_month_avg_amt  \
0                                  3                           0   
1                                  5                           1   
2                                  2                           0   
3                                  2                           1   
4                                  3                           0   
...                              ...                         ...   
89995                              7                           0   
89996                              2                           0   
89997                              3                           1   
89998                              5                           0   
89999                              9                           4   

       clnt_experience_cur_mnth  clnt_experience_cur_year  \
0                           0.0                       0.0   
1                           2.0                       0.0   
2                           0.0                       0.0   
3                           0.0                       0.0   
4                           0.0                       0.0   
...                         ...                       ...   
89995                       0.0                       0.0   
89996                       0.0                       0.0   
89997                       0.0                       0.0   
89998                       0.0                       0.0   
89999                       0.0                       0.0   

       clnt_experience_total_mnth  app_addr_region_reg  app_addr_region_fact  \
0                             0.0                  107                   107   
1                             3.0                    9                     9   
2                             0.0                  109                   109   
3                             0.0                   66                    66   
4                             0.0                   16                    16   
...                           ...                  ...                   ...   
89995                         0.0                    5                     5   
89996                         0.0                   56                    56   
89997                         1.0                  109                   109   
89998                         0.0                   60                    60   
89999                         0.0                   56                    56   

       app_addr_region_sale  clnt_birth_year  addr_region_fact_encoding1  \
0    

In [36]:
X = df[df.columns[2:]].to_numpy()
y = df['target'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [37]:
from catboost import CatBoostRegressor, CatBoostClassifier


model = CatBoostRegressor(iterations=10, learning_rate=1, task_type="GPU",
                           depth=10,loss_function="RMSE",eval_metric='AUC',
                           od_type = 'Iter', od_wait = 100, verbose = 10, l2_leaf_reg = 20)

model.fit(X_train, y_train, eval_set=(X_test, y_test))
dump(model, 'catboost.joblib')
roc_auc_score_at_K(reset_averages(m.predict(X_test)), y_test, rate=0.1)

0:	learn: 0.6930185	test: 0.6791714	best: 0.6791714 (0)	total: 18ms	remaining: 162ms
9:	learn: 0.7673515	test: 0.7050685	best: 0.7050685 (8)	total: 177ms	remaining: 0us
bestTest = 0.7050685058
bestIteration = 8
Shrink model to first 9 iterations.


0.854635891333372

In [26]:
m=load('best_catboost.joblib')

0.854635891333372

In [17]:
roc_auc_score_at_K(reset_averages(model.predict(X_test)), y_test, rate=0.1)

0.854635891333372

In [13]:
yproba = model.predict(X_test)
target = y_test
reset_averages(yproba)

order = np.argsort(-yproba)
top_k = int(0.10 * len(yproba))
yproba = yproba[order][:top_k]
target = target[order][:top_k]

fpr, tpr, _ = roc_curve(target,  yproba)
auc = roc_auc_score(target, yproba)
fig = plt.figure(figsize=(8,6))

plt.plot(fpr,          tpr, 
         label="AUC={:.3f}".format(auc))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')



plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("Ложноположительный", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("Истинноположительный", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

In [243]:
sum(predicted_proba[order][:top_k])

2557.750574995618

In [244]:
sum(predicted_proba[order][:top_k])

2557.750574995618

In [158]:
predict=model.predict(X)
top_k = int(0.1 * len(predict))
predict[np.argsort(-predict)][:top_k]

array([1.1782304 , 1.17511986, 1.17363734, ..., 0.78098059, 0.78095303,
       0.78095251])

In [130]:
df['predict'] = model.predict(X)

In [147]:
df[df.predict >= 0.78095251][df.target == 0]

card_id  target  addr_region_reg  addr_region_fact  \
30      cid_51279       0               37                37   
88     cid_143359       0               37                37   
108     cid_22737       0               63                63   
117     cid_42049       0               93                93   
145     cid_50445       0              101               101   
...           ...     ...              ...               ...   
89774   cid_19804       0              109               109   
89838   cid_91135       0               48               123   
89861   cid_33595       0               94                93   
89892  cid_105624       0               28                28   
89910  cid_130156       0               92                92   

       sas_limit_after_003_amt  sas_limit_last_amt  \
30                           3                   3   
88                           2                   2   
108                          0                   0   
117                          9                   9   
145                          0                   0   
...                        ...                 ...   
89774                        3                   3   
89838                        4                   4   
89861                        6                   6   
89892                        2                   2   
89910                        1                   1   

       clnt_income_month_avg_net_amt  clnt_expense_month_avg_amt  \
30                                 3                           0   
88                                 1                           0   
108                                2                           0   
117                                3                           0   
145                                1                           0   
...                              ...                         ...   
89774                              2                           0   
89838                              2                           1   
89861                              5                           1   
89892                              1                           0   
89910                              1                           0   

       clnt_experience_cur_mnth  clnt_experience_cur_year  \
30                          0.0                       0.0   
88                          2.0                       0.0   
108                         1.0                       0.0   
117                         0.0                       0.0   
145                         0.0                       0.0   
...                         ...                       ...   
89774                       1.0                       0.0   
89838                       0.0                       0.0   
89861                       0.0                       0.0   
89892                       0.0                       0.0   
89910                       3.0                       0.0   

       clnt_experience_total_mnth  app_addr_region_reg  app_addr_region_fact  \
30                            2.0                   37                    37   
88                            4.0                   37                    37   
108                           2.0                   63                    63   
117                           0.0                   93                    93   
145                           0.0                  101                   101   
...                           ...                  ...                   ...   
89774                         1.0                  109                   109   
89838                         0.0                   48                   123   
89861                         1.0                   94                    93   
89892                         0.0                   28                    28   
89910                         4.0                   92                    92   

       app_addr_region_sale  clnt_birth_year  addr_region_fact_encoding1  \
30   

In [148]:
df[df.predict >= 0.78095251][df.target == 1]

card_id  target  addr_region_reg  addr_region_fact  \
20      cid_29520       1               12                12   
38     cid_110174       1               92                92   
56      cid_34450       1               60                60   
57      cid_99408       1                3               100   
84      cid_35111       1               37                37   
...           ...     ...              ...               ...   
89912   cid_96204       1              106               106   
89916  cid_111524       1               20                20   
89949    cid_9735       1               20                20   
89971  cid_142055       1              101               101   
89982   cid_45858       1              107               107   

       sas_limit_after_003_amt  sas_limit_last_amt  \
20                           2                   2   
38                           1                   1   
56                           2                   2   
57                           9                   9   
84                           0                   0   
...                        ...                 ...   
89912                        0                   0   
89916                        5                   5   
89949                        1                   1   
89971                        0                   0   
89982                        4                   4   

       clnt_income_month_avg_net_amt  clnt_expense_month_avg_amt  \
20                                 3                           0   
38                                 2                           0   
56                                 3                           2   
57                                 5                           0   
84                                 2                           0   
...                              ...                         ...   
89912                              1                           0   
89916                              2                           0   
89949                              1                           1   
89971                              3                           0   
89982                              1                           0   

       clnt_experience_cur_mnth  clnt_experience_cur_year  \
20                          4.0                       0.0   
38                          0.0                       0.0   
56                          0.0                       0.0   
57                          0.0                       0.0   
84                          0.0                       0.0   
...                         ...                       ...   
89912                       0.0                       0.0   
89916                       1.0                       0.0   
89949                       8.0                       0.0   
89971                       2.0                       0.0   
89982                       0.0                       0.0   

       clnt_experience_total_mnth  app_addr_region_reg  app_addr_region_fact  \
20                            2.0                   12                    12   
38                            1.0                   92                    92   
56                            0.0                   60                    60   
57                            0.0                    3                   100   
84                            0.0                   37                    37   
...                           ...                  ...                   ...   
89912                         2.0                  106                   106   
89916                         2.0                   20                    20   
89949                         5.0                   20                    20   
89971                         1.0                  101                   101   
89982                         0.0                  107                   107   

       app_addr_region_sale  clnt_birth_year  addr_region_fact_encoding1  \
20   

In [161]:
pd.set_option('display.max_row', 500)
df[df.predict >= 0.78095251].corr()

target  addr_region_reg  \
target                              1.000000         0.010631   
addr_region_reg                     0.010631         1.000000   
addr_region_fact                    0.013875         0.955220   
sas_limit_after_003_amt            -0.005894        -0.005794   
sas_limit_last_amt                 -0.005916        -0.005322   
clnt_income_month_avg_net_amt       0.007742         0.013498   
clnt_expense_month_avg_amt         -0.004237        -0.008984   
clnt_experience_cur_mnth            0.061047        -0.012898   
clnt_experience_cur_year            0.004548        -0.017625   
clnt_experience_total_mnth          0.042549         0.016347   
app_addr_region_reg                 0.010624         1.000000   
app_addr_region_fact                0.010770         0.962574   
app_addr_region_sale               -0.024810         0.107510   
clnt_birth_year                     0.004282         0.039022   
addr_region_fact_encoding1         -0.024979        -0.010650   
addr_region_fact_encoding2         -0.026983        -0.022135   
addr_region_reg_encoding1          -0.008703        -0.003572   
addr_region_reg_encoding2          -0.010677        -0.017527   
app_addr_region_reg_encoding1      -0.008627        -0.003593   
app_addr_region_reg_encoding2      -0.010726        -0.017513   
app_addr_region_fact_encoding1     -0.024564        -0.010188   
app_addr_region_fact_encoding2     -0.026421        -0.021840   
app_addr_region_sale_encoding1      0.019238         0.080769   
app_addr_region_sale_encoding2      0.018832         0.061724   
loans_main_borrower                 0.007501         0.001884   
loans_active                       -0.038898        -0.010226   
last_loan_date                      0.007214         0.016299   
first_loan_date                     0.033660         0.009742   
max_overdue_status                 -0.006996        -0.000296   
ttl_officials                      -0.008702        -0.009290   
ttl_legals                          0.007293         0.010912   
ttl_bankruptcies                         NaN              NaN   
inquiry_recent_period              -0.010610         0.006031   
inquiry_3_month                    -0.008928        -0.050469   
inquiry_6_month                    -0.019265        -0.045367   
inquiry_9_month                    -0.022779        -0.046787   
inquiry_12_month                   -0.031165        -0.034900   
ttl_inquiries                      -0.024882        -0.031897   
ttl_auto_loan                      -0.013125         0.026670   
ttl_mortgage                        0.026278         0.003101   
ttl_credit_card                    -0.011210        -0.027204   
ttl_consumer                        0.013473         0.022125   
worst_status_ever                  -0.025428         0.007342   
feature_1                          -0.012701        -0.003933   
feature_2                          -0.013274        -0.004755   
feature_3                          -0.011737         0.006580   
feature_4                          -0.016712         0.019202   
feature_5                          -0.015110         0.007025   
feature_6                          -0.005710         0.004088   
feature_7                          -0.001087         0.007666   
feature_8                           0.008221         0.002781   
feature_9                          -0.044345        -0.004018   
fl_coborrower                       0.026451         0.005777   
fl_active_coborrower                0.029317         0.006926   
pay_load                           -0.027939        -0.003819   
inquiry_1_week                      0.007838        -0.046058   
inquiry_1_month                     0.001706        -0.046099   
feature_10                         -0.013214        -0.038425   
feature_12                         -0.010004        -0.040619   
feature_13                         -0.012485        -0.044551   
feature_14                         -0.013819        -0.018596   
feature_

In [242]:
predict=model.predict(X_test)
sns.distplot(predict[y_test==0])
sns.distplot(predict[y_test==1])

<AxesSubplot:ylabel='Density'>

In [45]:
import xgboost as xgb
from sklearn import metrics




# Parameter Tuning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}
param_dist['gpu_id'] = 0
param_dist['tree_method'] = 'gpu_hist'
"""
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(X, y)

grid_search.best_estimator_
"""
model = xgb.XGBRegressor(max_depth=50, min_child_weight=1,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(X_train,y_train)

[09:31:50] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.16, max_delta_step=0, max_depth=50,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbose=1,
             verbosity=None)

In [101]:
model.predict(X_test)

array([0.61395508, 0.6961838 , 0.17061862, ..., 0.26960611, 0.50190867,
       0.56025117])

In [10]:
from sklearn.metrics import roc_auc_score

In [102]:
predicted_proba = model.predict(X_test)
target = y_test

order = np.argsort(-predicted_proba)
top_k = int(0.1 * len(predicted_proba))

target[order][:top_k]
predicted_proba[order][:top_k]

array([1.7637108 , 1.64660753, 1.64533695, ..., 0.9231119 , 0.92305387,
       0.92299957])

In [103]:
roc_auc_score(target[order][:top_k], predicted_proba[order][:top_k])

0.5406010203806657

In [111]:
sum(target[order][:top_k]==1)/top_k

0.7094276094276094

In [112]:
sum(target[order][:top_k]==0)/top_k

0.2905723905723906

In [310]:
df=pd.concat([df.iloc[:,13:23], df.iloc[:,1]], axis=1)

In [221]:
df['t_0']=(df['addr_region_fact_encoding1']+df['addr_region_fact_encoding2'])
df['t_1']=(df['addr_region_reg_encoding1']+df['addr_region_reg_encoding2'])
df['t_2']=(df['app_addr_region_reg_encoding2']+df['app_addr_region_reg_encoding1'])
df['t_3']=(df['app_addr_region_fact_encoding2']+df['app_addr_region_fact_encoding1'])
df['t_4']=(df['app_addr_region_sale_encoding2']+df['app_addr_region_sale_encoding1'])
t_list=['t_0', 't_1', 't_2', 't_3', 't_4']
#df=df.iloc[:,-5:]
unique_list = np.unique(df.t_0.to_list()+df.t_1.to_list()+df.t_2.to_list()+df.t_3.to_list()+df.t_4.to_list())

In [222]:
df[t_list].value_counts()

t_0     t_1     t_2     t_3     t_4   
680000  680000  680000  680000  182000    5990
940000  940000  940000  940000  460000    5978
910000  910000  910000  910000  364000    4951
822500  822500  822500  822500  364000    3031
940000  940000  940000  940000  364000    3004
                                          ... 
702500  738750  738750  702500  364000       1
        702500  702500  702500  332000       1
                                182000       1
                                132000       1
0       0       0       0       90000        1
Length: 1853, dtype: int64

In [227]:
df[df.t_1==680000][t_list[:-1]].value_counts()

t_0     t_1     t_2     t_3   
680000  680000  680000  680000    8703
940000  680000  680000  940000     180
910000  680000  680000  910000     116
822500  680000  680000  822500      48
668750  680000  680000  668750      32
817500  680000  680000  680000      26
795000  680000  680000  795000      26
745000  680000  680000  745000      23
817500  680000  680000  817500      13
738750  680000  680000  738750      11
670000  680000  680000  670000       8
633750  680000  680000  633750       5
702500  680000  680000  702500       5
638750  680000  680000  638750       4
576250  680000  680000  576250       4
592500  680000  680000  592500       3
601250  680000  680000  601250       3
616250  680000  680000  616250       3
746250  680000  680000  746250       3
757500  680000  680000  757500       3
713750  680000  680000  713750       2
542500  680000  680000  542500       2
748750  680000  680000  748750       2
557500  680000  680000  557500       2
701250  680000  680000  701250   

In [423]:
unique_list = np.unique(df.t_0.to_list()+df.t_1.to_list()+df.t_2.to_list()+df.t_3.to_list())
len(unique_list)

68

In [230]:
df["_"]=(df.t_0==680000)&(df.t_1==680000)&(df.t_2==680000)&(df.t_3==680000)*1

In [232]:
df.corr()['_']

target                                0.022808
addr_region_reg                      -0.195847
addr_region_fact                     -0.195003
sas_limit_after_003_amt              -0.133515
sas_limit_last_amt                   -0.133342
clnt_income_month_avg_net_amt        -0.110270
clnt_expense_month_avg_amt           -0.069098
clnt_experience_cur_mnth             -0.030429
clnt_experience_cur_year             -0.001091
clnt_experience_total_mnth           -0.083682
app_addr_region_reg                  -0.195726
app_addr_region_fact                 -0.203686
app_addr_region_sale                 -0.017945
clnt_birth_year                       0.078076
addr_region_fact_encoding1           -0.176983
addr_region_fact_encoding2           -0.185721
addr_region_reg_encoding1            -0.129052
addr_region_reg_encoding2            -0.140790
app_addr_region_reg_encoding1        -0.128990
app_addr_region_reg_encoding2        -0.140737
app_addr_region_fact_encoding1       -0.173069
app_addr_regi

In [449]:
df = preprocess(df)

In [14]:
X = df[df.columns[2:]].to_numpy()
y = df['target'].to_numpy()

In [15]:
lr = LR()
lr.create_model(LR.default_model_constructor_parameters)
lr.fit_model(X, y, test_size=0.0)
lr.save_model()
roc_auc_score_at_K(lr.predict(X).reshape(-1), y, rate=0.1)

0.5571704230641429

In [458]:
predict = lr.predict(X).reshape(-1)

In [459]:

top_k = int(0.1 * len(predict)) 


order = np.argsort(-predict)
roc_auc_score(y[order][:top_k], predict[order][:top_k])

0.5557346352601609